In [22]:
import hdf5storage
import glob, os
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

days = ['20210911', '20210912', '20210913', '20210914', '20210915', '20210916', '20210917', '20210918', '20210919', '20210920', '20210921', '20210922', '20210923', '20210924', '20210925', '20210926', '20210927', '20210928', '20210929', '20210930', '20211001', '20211002', '20211003', '20211004', '20211005', '20211006', '20211007', '20211008']
fish_keys = ['23442333_back', '23484201_back', '23484204_back', '23520257_back', '23520258_back', '23520264_back', '23520266_back', '23520268_back', '23520270_back', '23520276_back', '23520278_back', '23520289_back', '23442333_front', '23484201_front', '23484204_front', '23520257_front', '23520258_front', '23520264_front', '23520266_front', '23520268_front', '23520270_front', '23520276_front', '23520278_front', '23520289_front']
projectPath = "/Volumes/Extreme_SSD/WhereEverYouDownloadTheProjectionData"
BLOCK = "block1"
def load_trajectory_data(projectPath,fish_key="", day=""):
    data_by_day = []
    pfile = glob.glob(projectPath+f'/Projections/{BLOCK}_{fish_key}*_{day}*_pcaModes.mat')
    pfile.sort()
    for f in pfile: 
        data = hdf5storage.loadmat(f)
        data_by_day.append(data)
    return data_by_day

def load_summerized_data(projectPath, fish_key="", day=""):
    proj_data = load_trajectory_data(projectPath, fish_key, day=day)
    positions = np.concatenate([trj["positions"] for trj in proj_data])
    projections = np.concatenate([trj["projections"] for trj in proj_data])
    daily_df = 5*(60**2)*8
    df_time_index = np.concatenate([trj["df_time_index"].flatten()+(daily_df*days.index(trj["day"].flatten()[0].split("_")[0])) for trj in proj_data])
    return dict(positions=positions, projections=projections, df_time_index=df_time_index)

def plasticity_cv_fig(fish_keys, filepath=None,n_df=50, forall=False):
    """
    calculates und plots the coefficient of variation 
    @fish_keys - a list of fish_keys for which to calculate plasticity
    - optional arguments
        @filepath - if and where the figure should be saved
        @n_df     - the number of data frames the CV is calculated for i.e. 1, 10, 50
        @forall   - a boolean to indicate where to compute the liniarregression for all fish_keys or for each. 
    returns: a figure 
    """
    fig = plt.figure(figsize=(10,10))
    axes = fig.subplots(3,1)
    fig.suptitle("Coefficient of Variation for %0.1f sec data"%(n_df/5))
    names = ["step length", "turning angle", "distance to wall"]
    colors_map = plt.cm.get_cmap(lut=len(fish_keys))
    sum_data = [{'time': [], 'cv': []}, {'time': [], 'cv': []}, {'time': [], 'cv': []}]
    for j, fk in enumerate(fish_keys):
        sum_data_fk = load_summerized_data(projectPath, fish_key=fk)
        data = sum_data_fk["projections"]
        time_idx = sum_data_fk["df_time_index"].flatten()
        for i, ax in enumerate(axes):
            data_in = data[:,i] if i != 1 else data[:,i]+np.pi # shift for turning angle 
            new_len = data_in.size//n_df
            data_in = data_in[:n_df*new_len].reshape((new_len, n_df)).mean(axis=1) # compute the means for each n_df
            n_means = (5*(60**2))//n_df
            sw_len = data_in.size//n_means
            sw_view = data_in[:sw_len*n_means].reshape(sw_len,n_means)
            cv = sw_view.std(axis=1)/sw_view.mean(axis=1) # compute means and std over one hour
            time = time_idx[::n_df][:sw_len*n_means:n_means]
            corrcof = pearsonr(time,cv) 
            if forall:
                sum_data[i]["time"].append(time)
                sum_data[i]["cv"].append(cv)
            ax.plot(time, cv, "o",color=colors_map(j), label="%s - pearsonr: %0.3f"%(fk[6:10], corrcof[0]) if not forall else None)
            ax.set_ylabel(names[i])
            if not forall: 
                a,c = np.polyfit(time,cv, 1)
                ax.plot(time, a*time+c, "-", color=colors_map(j))
                ax.legend(ncol=1)
    if forall:
        for i, ax in enumerate(axes):
            time = np.concatenate(sum_data[i]["time"])
            cv = np.concatenate(sum_data[i]["cv"])
            corrcof = pearsonr(time,cv)
            a,b,c = np.polyfit(time,cv, 2)
            t_range = np.arange(0, time.max())
            ax.plot(t_range, a*(t_range**2)+b*t_range+c, "-", color="k", markersize=12, label="pearsonr: %0.3f"%(corrcof[0]))
            ax.legend(ncol=1)
    ax.set_xlabel("tracked data frames")
    fig.tight_layout()
    if filepath:
        fig.savefig(f"{filepath}.pdf")
    return fig

dirname = f"plasticity"
os.makedirs(dirname, exist_ok=True)

In [ ]:
# calculates our plots for 1, 10, 50 mean data frames 
for n_df in [1,10,50]:
    plasticity_cv_fig(fish_keys[:-1], 
                filepath=f"{dirname}/plasticity_cv_all_df_%s_poly2"%(n_df),
                n_df=n_df, forall=True)